In [ ]:
"""
Example script demonstrating the portfolio optimization and backtesting system.

This script shows how to:
1. Load data from CSV files
2. Create and configure optimization strategies
3. Run backtests
4. Compare strategy performance
5. Generate reports and visualizations
"""

import pandas as pd
import numpy as np
from datetime import datetime
import tempfile
import os
from pathlib import Path

# Import our modules
from optfolio.data.loader import DataLoader
from optfolio.data.validator import DataValidator
from optfolio.backtesting.engine import Backtester
from optfolio.strategies.base import StrategyFactory

: 

In [2]:
data_dir = "price"

# Step 2: Load and validate data
print("\n1. Loading and validating data...")
data_loader = DataLoader(data_dir)
sectors = [
        "XLE", "XLF", "XLU", "XLI", "GDX", "XLK", "XLV", "XLY", "XLP", "XLB",
        "XOP", "IYR", "XHB", "ITB", "VNQ", "GDXJ", "IYE", "OIH", "XME", "XRT",
        "SMH", "IBB", "KBE", "KRE", "XTL"
    ]
sectors = ['WSM', 'PAYX', 'BMI', 'BK', 'NDAQ', 'MSI', 'WMT', 'TJX', 'AIG', 'RJF', 'V', 'CTAS', 'TT', 'TRGP', 'JPM', 'GE', 'MCK', 'PH', 'LLY', 'COST', 'AVGO', 'NEE', 'AMAT', 'ADI', 'SHW', 'INTU', 'KLAC']
# Load all available data
prices = data_loader.load_prices(tickers=sectors)
returns = data_loader.get_returns()

print(f"   Loaded {len(prices.columns)} tickers")
print(prices.columns)
print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
print(f"   Total observations: {len(prices)}")

# Validate data
validator = DataValidator()
price_issues = validator.validate_price_data(prices)
returns_issues = validator.validate_returns_data(returns)

print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")

if price_issues['warnings']:
    print(f"   Warnings: {len(price_issues['warnings'])}")
if price_issues['errors']:
    print(f"   Errors: {len(price_issues['errors'])}")


1. Loading and validating data...


Loading price data: 100%|██████████| 27/27 [00:00<00:00, 304.30it/s]

   Loaded 27 tickers
Index(['AMAT', 'GE', 'AIG', 'PH', 'MCK', 'LLY', 'CTAS', 'RJF', 'WMT', 'PAYX',
       'BK', 'TT', 'AVGO', 'BMI', 'NEE', 'WSM', 'MSI', 'TJX', 'TRGP', 'ADI',
       'V', 'JPM', 'INTU', 'NDAQ', 'KLAC', 'SHW', 'COST'],
      dtype='object')
   Date range: 2015-09-02 00:00:00 to 2025-09-01 00:00:00
   Total observations: 2513
   Data quality score: 1.000
   Warnings: 1


In [5]:
print("\n2. Creating optimization strategies...")

strategies = [
    StrategyFactory.create('equal_weight'),
    StrategyFactory.create('mean_variance', objective="sortino_ratio"),
    StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
    StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
    # StrategyFactory.create('black_litterman', 
    #                         prior_method="market_cap", view_method="momentum"),
    StrategyFactory.create('black_litterman', prior_method="market_cap", view_method="upside")
]

strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
for i, strategy in enumerate(strategies):
    strategy.name = strategy_names[i]
    print(f"   Created: {strategy}")


2. Creating optimization strategies...
   Created: Equal Weight Strategy (Equal Weight)
   Created: Mean-Variance (Sortino) Strategy (sortino_ratio)
   Created: Mean-Variance (Sharpe) Strategy (sharpe_ratio)
   Created: Random Weight Strategy (dirichlet)
   Created: Black-Litterman Strategy (market_cap priors, upside views)


In [6]:
# Step 4: Set up backtesting
print("\n3. Setting up backtesting...")

backtester = Backtester(
    initial_capital=100000.0,
    risk_free_rate=0.03,
    transaction_costs=0.001
)

# Load data into backtester
backtester.load_data(data_loader, tickers=sectors)


3. Setting up backtesting...


Loading price data: 100%|██████████| 27/27 [00:00<00:00, 247.45it/s]

Loaded data for 27 tickers
Date range: 2015-09-02 00:00:00 to 2025-09-01 00:00:00
Total observations: 2513


In [7]:
# Step 5: Run backtests
print("\n4. Running backtests...")

# Define rebalancing schedule (every 3 months, 1st week, 1st day)
rebalance_freq = {"months": 3}

# Run backtests for all strategies
results = backtester.run_multiple_backtests(
    strategies=strategies,
    rebalance_freq=rebalance_freq,
    start_date='2016-01-01',  # Start from 2021 to have enough history
    end_date='2024-12-31'
)

all_results = []
for strategy_name, result in results.items():
    metrics = result['performance_metrics']
    summary = result['summary']
    
    # Collect metrics for each strategy in a list of dicts
    all_results.append({
        "Strategy": strategy_name,
        "Total Return": metrics.get('total_return', 0),
        "Annualized Return": metrics.get('annualized_return', 0),
        "Volatility": metrics.get('volatility', 0),
        "Sharpe Ratio": metrics.get('sharpe_ratio', 0),
        "Sortino Ratio": metrics.get('sortino_ratio', 0),
        "Max Drawdown": metrics.get('max_drawdown', 0),
        "Number of Transactions": summary.get('num_transactions', 0),
        "Total Transaction Costs": summary.get('total_transaction_costs', 0),
        # "T-Statistic": backtester.significance(strategy_name).get('t_statistic', np.nan),
        "P-Value": backtester.significance(strategy_name).get('p_value', np.nan)
    })
resultsdf = pd.DataFrame(all_results)
resultsdf 


4. Running backtests...
Running backtest for Equal Weight
Rebalancing dates: 29
Date range: 2016-01-03 00:00:00 to 2024-12-30 00:00:00
Running backtest for Mean-Variance (Sortino)
Rebalancing dates: 29
Date range: 2016-01-03 00:00:00 to 2024-12-30 00:00:00
Running backtest for Mean-Variance (Sharpe)
Rebalancing dates: 29
Date range: 2016-01-03 00:00:00 to 2024-12-30 00:00:00
Running backtest for Random Weight
Rebalancing dates: 29
Date range: 2016-01-03 00:00:00 to 2024-12-30 00:00:00
Running backtest for Black-Litterman
Rebalancing dates: 29
Date range: 2016-01-03 00:00:00 to 2024-12-30 00:00:00


/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

...

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/share/home/michael/projects/optfolio/optfolio/strategies/upside.py:72: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,Strategy,Total Return,Annualized Return,Volatility,Sharpe Ratio,Sortino Ratio,Max Drawdown,Number of Transactions,Total Transaction Costs,P-Value
0,Equal Weight,4.956277,0.221616,1.397088,5.348469,13.882709,-0.145632,28,673.873950,0.003434
1,Mean-Variance (Sortino),4.706651,0.215763,1.084086,6.485117,14.897286,-0.109089,28,2411.802012,0.000600
2,Mean-Variance (Sharpe),4.956277,0.221616,1.397088,5.348469,13.882709,-0.145632,28,673.873950,0.003434
3,Random Weight,4.856766,0.219309,1.444841,5.155080,14.820096,-0.140043,28,6872.223563,0.004522
4,Black-Litterman,5.064618,0.224089,1.708466,4.638602,9.715828,-0.207178,28,8543.793209,0.009150


In [ ]:
results 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
strat = list(results.keys())[4]
# Assuming you have the weight history from your backtest result
weight_df = pd.DataFrame(results[strat]['weight_history'])

# Create the plot
plt.figure(figsize=(15, 10))

# Plot each asset's weight over time
for asset in weight_df.columns:
    plt.plot(weight_df.index, weight_df[asset], label=asset, linewidth=2)

plt.title(f'Asset Allocation Over Time - {strat}', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Weight', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Stacked area chart
plt.figure(figsize=(15, 10))
weight_df.plot(kind='area', stacked=True, figsize=(15, 10))
plt.title('Asset Allocation Over Time (Stacked)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Weight', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

# Heatmap
plt.figure(figsize=(15, 8))
sns.heatmap(weight_df.T, 
            annot=False,           # Don't show values (too cluttered)
            fmt='.2f',           
            cmap='RdYlBu_r',     # Color map
            center=0.5,          # Center color scale
            cbar_kws={'label': 'Weight'})

plt.title('Asset Allocation Heatmap Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Asset', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
strategy = StrategyFactory.create('black_litterman', 
                            prior_method="equal", view_method="momentum")
rebalance_freq = {"months": 3}                            
result = backtester.run_backtest(strategy=strategy,rebalance_freq=rebalance_freq,
    start_date='2018-01-01',  # Start from 2021 to have enough history
    end_date='2023-12-31')

In [ ]:
result['rolling_metrics']

In [ ]:
metrics = result['performance_metrics']
summary = result['summary']
print(f"\n{strategy}:")
print(f"  Total Return: {metrics.get('total_return', 0):.2%}")
print(f"  Annualized Return: {metrics.get('annualized_return', 0):.2%}")
print(f"  Volatility: {metrics.get('volatility', 0):.2%}")
print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
print(f"  Sortino Ratio: {metrics.get('sortino_ratio', 0):.3f}")
print(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
print(f"  P-Value: {backtester.significance('black_litterman').get('p_value', np.nan)}")
print(f"Summary: {result['summary']}")

In [ ]:
# pip install arch
from arch.bootstrap import StationaryBootstrap

# Your daily returns series
# returns = vbt_portfolio.returns()
returns = result['returns']

# We need to define a function to calculate the Sharpe ratio
# Note: quantstats annualizes it, so we do it manually here for the bootstrap
def annualized_sharpe(r):
    # Assuming 252 trading days
    return (r.mean() / r.std()) * np.sqrt(252)

# Set up the bootstrap
# Use a block size appropriate for your data's autocorrelation. For daily returns, 5-10 is common.
bs = StationaryBootstrap(block_size=10, x=returns)

# Run 10,000 simulations and calculate the Sharpe ratio for each
results = bs.apply(annualized_sharpe, 10000)

# Create a confidence interval
confidence_interval = np.percentile(results, [2.5, 97.5])

print(f"Original Annualized Sharpe Ratio: {annualized_sharpe(returns):.2f}")
print(f"95% Confidence Interval for Sharpe Ratio: [{confidence_interval[0]:.2f}, {confidence_interval[1]:.2f}]")

if confidence_interval[0] > 0:
    print("\n✅ The 95% confidence interval is entirely above zero, suggesting robustness.")
else:
    print("\n⚠️ The 95% confidence interval includes zero, suggesting the true Sharpe ratio could be zero or negative.")

In [ ]:
# last weight
weight_df = pd.DataFrame(result['weight_history'])
print([result['last_weights']])
lw = pd.DataFrame([result['last_weights']]).T
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
lw.plot(kind='bar', ax=plt.gca())
plt.title('Last Weight Distribution')
plt.xlabel('Asset')
plt.ylabel('Weight')
plt.show()


In [ ]:
print(result['last_weights'])

In [ ]:
weight_df = pd.DataFrame(result['weight_history'])
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Create sample DataFrame


# Seaborn heatmap - works directly with DataFrame!
plt.figure(figsize=(12, 8))
sns.heatmap(weight_df, 
            annot=False,           # Show values
            fmt='.2f',           # Format to 2 decimal places
            cmap='RdYlBu_r',     # Color map
            center=0.5,          # Center color scale
            square=True,         # Make cells square
            cbar_kws={'label': 'Value'})

plt.title('DataFrame Heatmap (Seaborn)')
plt.tight_layout()
plt.show()

In [ ]:
#!/usr/bin/env python3



def run_portfolio_analysis():
    """Run the main portfolio analysis."""
    print("=" * 60)
    print("PORTFOLIO OPTIMIZATION AND BACKTESTING SYSTEM")
    print("=" * 60)
    
    # Step 1: Create sample data
    # data_dir = create_sample_data()
    data_dir = "price"
    
    # Step 2: Load and validate data
    print("\n1. Loading and validating data...")
    data_loader = DataLoader(data_dir)
    
    # Load all available data
    prices = data_loader.load_prices()
    returns = data_loader.get_returns()
    
    print(f"   Loaded {len(prices.columns)} tickers")
    print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
    print(f"   Total observations: {len(prices)}")
    
    # Validate data
    validator = DataValidator()
    price_issues = validator.validate_price_data(prices)
    returns_issues = validator.validate_returns_data(returns)
    
    print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")
    
    if price_issues['warnings']:
        print(f"   Warnings: {len(price_issues['warnings'])}")
    if price_issues['errors']:
        print(f"   Errors: {len(price_issues['errors'])}")
    
    # Step 3: Create optimization strategies
    print("\n2. Creating optimization strategies...")
    
    strategies = [
        StrategyFactory.create('equal_weight'),
        StrategyFactory.create('mean_variance', objective="sortino_ratio"),
        StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
        StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
        StrategyFactory.create('black_litterman', 
                             prior_method="market_cap", view_method="momentum")
    ]
    
    strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
    for i, strategy in enumerate(strategies):
        strategy.name = strategy_names[i]
        print(f"   Created: {strategy}")
    
    # Step 4: Set up backtesting
    print("\n3. Setting up backtesting...")
    
    backtester = Backtester(
        initial_capital=100000.0,
        risk_free_rate=0.02,
        transaction_costs=0.001
    )
    
    # Load data into backtester
    backtester.load_data(data_loader)
    
    # Step 5: Run backtests
    print("\n4. Running backtests...")
    
    # Define rebalancing schedule (every 3 months, 1st week, 1st day)
    rebalance_freq = {"months": 3, "weeks": 1, "days": 1}
    
    # Run backtests for all strategies
    results = backtester.run_multiple_backtests(
        strategies=strategies,
        rebalance_freq=rebalance_freq,
        start_date='2021-01-01',  # Start from 2021 to have enough history
        end_date='2023-12-31'
    )
    
    print(f"   Completed backtests for {len(results)} strategies")
    
    # Step 6: Compare strategies
    print("\n5. Comparing strategy performance...")
    
    comparison = backtester.compare_strategies()
    print("\nStrategy Performance Comparison:")
    print("-" * 80)
    print(comparison.round(3).to_string(index=False))
    
    # Step 7: Generate detailed analysis
    print("\n6. Detailed strategy analysis...")
    
    for strategy_name, result in results.items():
        metrics = result['performance_metrics']
        summary = result['summary']
        
        print(f"\n{strategy_name}:")
        print(f"  Total Return: {metrics.get('total_return', 0):.2%}")
        print(f"  Annualized Return: {metrics.get('annualized_return', 0):.2%}")
        print(f"  Volatility: {metrics.get('volatility', 0):.2%}")
        print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
        print(f"  Sortino Ratio: {metrics.get('sortino_ratio', 0):.3f}")
        print(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
        print(f"  Number of Transactions: {summary.get('num_transactions', 0)}")
        print(f"  Total Transaction Costs: ${summary.get('total_transaction_costs', 0):.2f}")
    
    # Step 8: Export results
    print("\n7. Exporting results...")
    
    # Export comparison to CSV
    comparison.to_csv('strategy_comparison.csv', index=False)
    print("   Exported strategy comparison to 'strategy_comparison.csv'")
    
    # Export detailed results to JSON
    backtester.export_results('detailed_results.json', format='json')
    print("   Exported detailed results to 'detailed_results.json'")
    
    # Step 9: Generate visualizations
    print("\n8. Generating visualizations...")
    
    try:
        import matplotlib.pyplot as plt
        
        # Plot portfolio values
        plt.figure(figsize=(12, 8))
        for strategy_name, result in results.items():
            if 'portfolio_values' in result and len(result['portfolio_values']) > 0:
                portfolio_series = result['portfolio_values']
                plt.plot(portfolio_series.index, portfolio_series.values, 
                        label=strategy_name, linewidth=2)
        
        plt.title('Portfolio Values Over Time')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value ($)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('portfolio_values.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   Saved portfolio values plot to 'portfolio_values.png'")
        
        # Plot rolling metrics
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        for i, (strategy_name, result) in enumerate(results.items()):
            rolling_metrics = result['rolling_metrics']
            
            if not rolling_metrics.empty:
                # Rolling Sharpe ratio
                axes[0, 0].plot(rolling_metrics.index, rolling_metrics['rolling_sharpe'], 
                               label=strategy_name, alpha=0.7)
                
                # Rolling volatility
                axes[0, 1].plot(rolling_metrics.index, rolling_metrics['rolling_volatility'], 
                               label=strategy_name, alpha=0.7)
                
                # Rolling return
                axes[1, 0].plot(rolling_metrics.index, rolling_metrics['rolling_return'], 
                               label=strategy_name, alpha=0.7)
                
                # Rolling drawdown
                axes[1, 1].plot(rolling_metrics.index, rolling_metrics['rolling_drawdown'], 
                               label=strategy_name, alpha=0.7)
        
        axes[0, 0].set_title('Rolling Sharpe Ratio')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        axes[0, 1].set_title('Rolling Volatility')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        axes[1, 0].set_title('Rolling Return')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        axes[1, 1].set_title('Rolling Drawdown')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        # plt.savefig('rolling_metrics.png', dpi=300, bbox_inches='tight')
        plt.show()
        plt.close()
        
        print("   Saved rolling metrics plot to 'rolling_metrics.png'")
        
    except ImportError:
        print("   Matplotlib not available, skipping visualizations")
    
    # Step 10: Summary
    print("\n" + "=" * 60)
    print("ANALYSIS COMPLETE")
    print("=" * 60)
    
    # Find best performing strategy
    if not comparison.empty and 'Sharpe Ratio' in comparison.columns:
        best_strategy = comparison.loc[comparison['Sharpe Ratio'].idxmax(), 'Strategy']
        best_sharpe = comparison['Sharpe Ratio'].max()
        
        print(f"\nBest performing strategy (by Sharpe ratio): {best_strategy}")
        print(f"Sharpe ratio: {best_sharpe:.3f}")
    else:
        print(f"\nNo successful backtests completed. Check the error messages above.")
    
    # Clean up temporary data
    import shutil
    shutil.rmtree(data_dir.parent)
    
    print(f"\nTemporary data cleaned up")
    print(f"\nGenerated files:")
    print(f"  - strategy_comparison.csv")
    print(f"  - detailed_results.json")
    print(f"  - portfolio_values.png")
    print(f"  - rolling_metrics.png")





In [ ]:
"""Run the main portfolio analysis."""
print("=" * 60)
print("PORTFOLIO OPTIMIZATION AND BACKTESTING SYSTEM")
print("=" * 60)

# Step 1: Create sample data
# data_dir = create_sample_data()
data_dir = "price"

# Step 2: Load and validate data
print("\n1. Loading and validating data...")
data_loader = DataLoader(data_dir)

# Load all available data
prices = data_loader.load_prices()
returns = data_loader.get_returns()

print(f"   Loaded {len(prices.columns)} tickers")
print(f"   Date range: {prices.index.min()} to {prices.index.max()}")
print(f"   Total observations: {len(prices)}")

# Validate data
validator = DataValidator()
price_issues = validator.validate_price_data(prices)
returns_issues = validator.validate_returns_data(returns)

print(f"   Data quality score: {validator.get_data_quality_score(prices):.3f}")

if price_issues['warnings']:
    print(f"   Warnings: {len(price_issues['warnings'])}")
if price_issues['errors']:
    print(f"   Errors: {len(price_issues['errors'])}")

# Step 3: Create optimization strategies
print("\n2. Creating optimization strategies...")

strategies = [
    StrategyFactory.create('equal_weight'),
    StrategyFactory.create('mean_variance', objective="sortino_ratio"),
    StrategyFactory.create('mean_variance', objective="sharpe_ratio"),
    StrategyFactory.create('random_weight', distribution="dirichlet", seed=42),
    StrategyFactory.create('black_litterman', 
                            prior_method="market_cap", view_method="momentum")
]

strategy_names = ["Equal Weight", "Mean-Variance (Sortino)", "Mean-Variance (Sharpe)", "Random Weight", "Black-Litterman"]
for i, strategy in enumerate(strategies):
    strategy.name = strategy_names[i]
    print(f"   Created: {strategy}")

# Step 4: Set up backtesting
print("\n3. Setting up backtesting...")

backtester = Backtester(
    initial_capital=100000.0,
    risk_free_rate=0.02,
    transaction_costs=0.001
)

# Load data into backtester
backtester.load_data(data_loader)

# Step 5: Run backtests
print("\n4. Running backtests...")

# Define rebalancing schedule (every 3 months, 1st week, 1st day)
rebalance_freq = {"months": 3, "weeks": 1, "days": 1}

# Run backtests for all strategies
results = backtester.run_multiple_backtests(
    strategies=strategies,
    rebalance_freq=rebalance_freq,
    start_date='2021-01-01',  # Start from 2021 to have enough history
    end_date='2023-12-31'
)

print(f"   Completed backtests for {len(results)} strategies")

# Step 6: Compare strategies
print("\n5. Comparing strategy performance...")

comparison = backtester.compare_strategies()
print("\nStrategy Performance Comparison:")
print("-" * 80)
print(comparison.round(3).to_string(index=False))

# Step 7: Generate detailed analysis
print("\n6. Detailed strategy analysis...")

for strategy_name, result in results.items():
    metrics = result['performance_metrics']
    summary = result['summary']
    
    print(f"\n{strategy_name}:")
    print(f"  Total Return: {metrics.get('total_return', 0):.2%}")
    print(f"  Annualized Return: {metrics.get('annualized_return', 0):.2%}")
    print(f"  Volatility: {metrics.get('volatility', 0):.2%}")
    print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.3f}")
    print(f"  Sortino Ratio: {metrics.get('sortino_ratio', 0):.3f}")
    print(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
    print(f"  Number of Transactions: {summary.get('num_transactions', 0)}")
    print(f"  Total Transaction Costs: ${summary.get('total_transaction_costs', 0):.2f}")

# Step 8: Export results
print("\n7. Exporting results...")